# Setup

In [26]:
! pip install langchain_community tiktoken langchain-ollama ollama langchainhub chromadb langchain bs4

  Using cached soupsieve-2.7-py3-none-any.whl.metadata (4.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 1.1 MB/s eta 0:00:00a 0:00:01
Using cached soupsieve-2.7-py3-none-any.whl (36 kB)


In [2]:
# Load env
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [3]:
! source get_host.sh && echo $OLLAMA_HOST > tmp_env.txt
with open('tmp_env.txt') as f:
    OLLAMA_HOST = f.read().strip()
%env OLLAMA_HOST=$OLLAMA_HOST
import os
os.remove("tmp_env.txt")

env: OLLAMA_HOST=http://172.25.208.1:11434


# Indexing

In [ ]:
# Documents
# question = "What kinds of pets do I like?"
# document = "My favorite pet is a cat."
question = "Have you eaten yet?"
document = "No I have not eaten yet."

In [5]:
import tiktoken

def num_tokens_from_string(string: str, encoding_name:str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

num_tokens_from_string(question, "cl100k_base")

8

In [23]:
# Text embedding models
# https://ollama.com/blog/embedding-models
from langchain_ollama import OllamaEmbeddings
# embd = OllamaEmbeddings(model="nomic-embed-text")
embd = OllamaEmbeddings(model="mxbai-embed-large")
query_result = embd.embed_query(question)
document_result = embd.embed_query(document)
len(query_result)

1024

In [24]:
import numpy as np

def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)

similarity = cosine_similarity(query_result, document_result)
print("Cosine Similarity:", similarity)


Cosine Similarity: 0.2761092814708602


In [30]:
# Document Loaders

# Load blog
import bs4
from langchain_community.document_loaders import WebBaseLoader
import os

os.environ['USER_AGENT'] = 'myagent'

loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
blog_docs = loader.load()

In [ ]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300,
    chunk_overlap=50,
)

# make splits
splits = text_splitter.split_documents(blog_docs)

# Retrieval

# Generation

In [4]:
from langchain.prompts import ChatPromptTemplate

# Prompt
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Answer the question based only on the following context:\n{context}\n\nQuestion: {question}\n'), additional_kwargs={})])

In [5]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="llama3.2:3b",
    base_url=os.getenv("OLLAMA_HOST")
)

In [6]:
chain = prompt | llm

In [7]:
docs = "The best animal in the world is the snake. After that is the chipmunk but only during a blue moon. If today is a weekday the best animal is the turtle."
chain.invoke({"context": docs, "question": "What is the best animal?"})

AIMessage(content="This question's answer depends on the day of the week and the lunar phase. Since the context doesn't specify the current date or time, we can only make an educated guess based on the given information.\n\nIf today is a weekday, the best animal is the turtle.\nHowever, if today is not a weekday (i.e., it's a weekend), the answer would be unclear because there's no specified criteria for what to do during a weekend.", additional_kwargs={}, response_metadata={'model': 'llama3.2:3b', 'created_at': '2025-08-20T07:47:43.1782357Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1506149800, 'load_duration': 37224700, 'prompt_eval_count': 79, 'prompt_eval_duration': 3307400, 'eval_count': 91, 'eval_duration': 1465109200, 'model_name': 'llama3.2:3b'}, id='run--5ca39d75-86e9-4e82-b4ca-f1a6b62c4be3-0', usage_metadata={'input_tokens': 79, 'output_tokens': 91, 'total_tokens': 170})